# Broadcasting of `numpy`-arrays

## Introduction

- *All* operations on `numpy`-arrays are by default element-wise.
- This trivially works for arrays with the same shape

In [ ]:
import numpy as np

a = np.arange(10)
b = np.arange(10, 20)

c = np.arange(25).reshape((5, 5))
d = np.arange(25, 50).reshape((5,5))

print(a, b)
print(a + b)   # elementwise addition of elements

print(c, d)
print(c * d)   # elementwise multiplication

print(a + d)   # Error: shapes do not match and the
               # addition is not defined!

- **However,** it is also possible to do `numpy`-operations on arrays of different
shapes *if* NumPy can transform these arrays to the same shape: this conversion is called **broadcasting**.
- You already used broadcasting in operations between arrays and scalars!

In [ ]:
import numpy as np

a = np.arange(4)

print(a)
print(a * 2)  # the '2' is broadcasted (stretched) to a (5,)-array

b = np.arange(16).reshape((4,4))

print(b)
print(b * 2) # the '2' is broadcasted (stretched) to a (5,5)-array
print(b + a) # the (5,)-array 'a' is broadcasted to a
             # (5,5)-array

## Practical Example: Bias correction of astronomical data

- Optical astronomical data are a two-dimensional array of pixel values
- Parts of the CCD is not illuminated during an exposure and hence provides *noisy* zero-level pixel values of an exposure. This part of a CCD is called *overscan region*
- You need to estimate an overscan value per column/row (depending on where the overscan region is; see figure below) and subtract that value from the corresponding column/row of your science data.

Effectively, we need to apply a lower-dimensional array (an overscan column/row) to a higher dimensional one (the science data). We somehow need to *stretch* the one-dimensional oversan values to a two-dimensional array so that an element-wise subtraction can be done.

This is a typical application of `numpy`-array broadcasting

![bias](figs/bias.png)

In [ ]:
import numpy as np
import numpy.random as nr

# make sure that 'random numbers' are reproducible in the following:
nr.seed(1)
# create fake data with some overscan region (horizontal overscan lines)
data_hor = nr.normal(loc=100, scale=1.0, size=40).reshape(10,4)
overscan_hor = nr.normal(loc=10, scale=1.0, size=8).reshape(2,4)
data_hor[-2:,:] = overscan_hor

print(data_hor)

# perform overscan correction (horizontal case)
overscan = data_hor[-2:,:].mean(axis=0)
data_corrected = data_hor - overscan
# to document better the broadcasting, the previous line
# is better written as
# data_corrected = data_hor - overscan[np.newaxis,:]

print(data_corrected)

In [ ]:
print(data_hor.shape, overscan.shape)

In [ ]:
import numpy as np
import numpy.random as nr

# make sure that 'random numbers' are reproducible in the following:
nr.seed(1)
# create fake data with some overscan region (vertical overscan column)
data_ver = nr.normal(loc=100, scale=1.0, size=40).reshape(10,4)
overscan_ver = nr.normal(loc=10, scale=1.0, size=20).reshape(10,2)
data_ver[:,:2] = overscan_ver

print(data_ver)

# perform overscan correction (vertical case)
overscan = data_ver[:,:2].mean(axis=1)
data_corrected = data_ver - overscan[:, np.newaxis]
print(data_corrected)

In [ ]:
print(data_ver.shape, overscan.shape)

## Formal definition of Broadcasting

- Braodcasting consists of a set of rules that permit *element-wise* operations of arrays of different shapes.
- Element-wise operations on arrays are only valid when the arrays' shapes are either equal or *compatible*.
- To determine if two shapes are compatible, `numpy` compares their dimensions, starting with the trailing ones and working its way backwards. If two dimensions are equal, or if one of them equals 1, the comparison continues. If one of the dimensions in this case is 1 and the other is larger than 1, the smaller array is stretched *naturally* to match the bigger one. Otherwise, you'll see a ValueError raised (saying something like "operands could not be broadcast together with shapes ...").
- When one of the shapes runs out of dimensions (because it has less dimensions than the other shape), `numpy` will use 1 in the comparison process until the other shape's dimensions run out as well.
- Once `numpy` determines that two shapes are compatible, the shape of the result is simply the maximum of the two shapes' sizes in each dimension.

**Examples:**

![braodcasting_1](figs/broadcasting_1.png)

**Important:**

The rules above are precise and complete!
- Note that missing dimensions in one array can only be *left-padded*!
- If you need to *right-pad* an array to make it compatible, you need to do this explicitely with a `np.newaxis` or a reshape command

![broadcase_2](figs/broadcasting_2.png)

In [ ]:
import numpy as np

a = np.arange(10).reshape(2, 5)
b = np.arange(2)

# print(a + b) would not work!!
# we need to manually 'right-pad' one dimension to
# 'b' to make it broadcast-compatible with a:
b = b.reshape((2, 1))
# b = b[:, np.newaxis] # equivalent to the reshape command above
print(a)
print(b)
print(a + b)

## Example: Outer product of two vectors 

The outer product of two vectors $x\in R^n$ and $y\in R^n$ is defined as:
$$
xy^T = \left(\begin{array}{c}x_1 \\ x_2 \\ \vdots \\ x_n \end{array}\right) \left(y_1, y_2, \dots, x_n\right)=
\left(\begin{array}{cccc}x_1y_1 & x_1y_2 & \dots & x_1 y_n \\
                         x_2y_1 & x_2y_2 & \dots & x_2 y_n \\
                         \vdots & \vdots & \ddots & \vdots \\
                         x_ny_1 & x_2y_n & \dots & x_n y_n \\
                         \end{array}\right).
$$
The resulting matrix is in $R^{nxn}$

In [ ]:
import numpy as np

# The outer product of two vectors can be done easily
# with a broadcasting operation
x = np.array([1, 2, 3])
y = np.array([4, 5, 6])

print(x, y)
print(x.shape, y.shape)

# solution:
outer_prod = x[:, np.newaxis] * y
print(outer_prod)